# Geostrophic adjustment using Oceananigans.ShallowWaterModel

See: https://clima.github.io/OceananigansDocumentation/stable/ and https://clima.github.io/OceananigansDocumentation/stable/generated/geostrophic_adjustment/

Geostrophic adjustment in a rectangle using the 1-layer RSW equations

# Install dependencies

 First let's make sure we have all required packages installed.

In [1]:
using Oceananigans
using Oceananigans.Units
using Oceananigans.Models: ShallowWaterModel
using Printf
export @printf, @sprintf, Printf
using Makie, GLMakie
using JLD2

# Define rectangular domain

We use a one-dimensional domain of geophysical proportions and geophysical parameters appropriate for the mid-latitudes on Earth.

In [2]:
#Lᵣ = 8kilometers           # Strong rotation, F = 4096
#Lᵣ = 32kilometers          # Moderate rotation
#Lᵣ = 64.05kilometers       # Weak rotation, F = 64
Lᵣ = 2*64.05kilometers       # Weak rotation, F = 16
#Lᵣ = 256.05kilometers      # Weak rotation, F = 4
#Lᵣ = 512kilometers         # Weak rotation, F = 1
f = 1e-4
g = 9.81
Δη = 0.001meters
H = ((f*Lᵣ)^2)/g
U = (Δη*g)/(Lᵣ*f)
Rₒ = U/(f*Lᵣ)
Lx = 512kilometers  # Let Lx >= Ly (wlog)
#Ly = 32kilometers
Ly = 512kilometers
gravity_wave_speed = sqrt(g * H) # hydrostatic (shallow water) gravity wave speed
timescale = Lx/gravity_wave_speed
damping_timescale = 2e6/f  # Dimensional damping time

# Nx, Ny, Nt = 128,128,2048
Nx, Ny, Nt = 128,128,4096
#Nx, Ny, Nt = 256,256,2048
#Nx, Ny, Nt = 128,16,4096

damping = Relaxation(rate = 1/damping_timescale)

grid = RectilinearGrid(size = (Nx, Ny),
                              x = (-Lx/2, Lx/2), y = (-Ly/2, Ly/2),
                              topology = (Bounded, Bounded, Flat)) 
@printf("Channel length            = [%2.2f] km \n", Lx/1e3)
@printf("Channel width             = [%2.2f] km \n", Ly/1e3)
@printf("Deformation radius        = [%3.2f] km \n", Lᵣ/1e3)
@printf("Layer thickness           = [%2.4f] m \n", H)
@printf("Grid resolution           = [%2.0d x %2.0d x %2.0d] [Nx x Ny x Nt]\n", Nx, Ny, Nt)
@printf("Δη                        = [%2.4f] m \n", Δη)
@printf("Velocity scale            = [%2.4f] m s⁻¹\n", U)
@printf("Gravity wave speed        = [%2.4f] m s⁻¹\n", gravity_wave_speed)
@printf("Damping timescale         = [%2.4f] days \n", damping_timescale / days)
@printf("Rossby number             = [%2.4f] \n", Rₒ)
@printf("Inverse Froude number     = [%8.2f]\n",Lx^2/Lᵣ^2)
@printf("Non-dim. deformation rad. = [%8.2f]\n",Lᵣ/Lx)
@printf("Timescale                 = [%8.2f] days\n",timescale/86400)

Channel length            = [512.00] km 
Channel width             = [512.00] km 
Deformation radius        = [128.10] km 
Layer thickness           = [16.7274] m 
Grid resolution           = [128 x 128 x 4096] [Nx x Ny x Nt]
Δη                        = [0.0010] m 
Velocity scale            = [0.0008] m s⁻¹
Gravity wave speed        = [12.8100] m s⁻¹
Damping timescale         = [231481.4815] days 
Rossby number             = [0.0001] 
Inverse Froude number     = [   15.98]
Non-dim. deformation rad. = [    0.25]
Timescale                 = [    0.46] days


# Build a `ShallowWaterModel`

In [3]:
model = ShallowWaterModel(
    timestepper=:RungeKutta3,
    advection=WENO5(),
    grid=grid,
    gravitational_acceleration=g,
    coriolis=FPlane(f=f),
    forcing=(uh=damping, vh=damping))

┌ Warning: defaulting to uniform WENO scheme with Float64 precision, use WENO5(grid = grid) if this was not intended
└ @ Oceananigans.Advection /Users/thaine1/.julia/packages/Oceananigans/PA1TE/src/Advection/weno_fifth_order.jl:144


ShallowWaterModel{typename(CPU), Float64}(time = 0 seconds, iteration = 0) 
├── grid: 128×128×1 RectilinearGrid{Float64, Bounded, Bounded, Flat} on CPU with 3×3×0 halo
├── tracers: ()
└── coriolis: FPlane{Float64}

# A geostrophic adjustment initial value problem

We pose a geostrophic adjustment problem that consists of an initial hyperbolic tangent height field, resembling the Rossby step

In [4]:
Tanh(x, L) = tanh(x/L)

#ΔL = Lx / (Nx/4)  # tanh width
ΔL = Lx / 8  # tanh width
hⁱ(x, y, z) = H + Δη*Tanh(x,ΔL)

set!(model, h=hⁱ)
uh, vh, h = model.solution
        # η = ComputedField((h - H)/Δη)                  # Non-dimensional
        # u = ComputedField((uh / h)/U)                  # Non-dimensional
        # v = ComputedField((vh / h)/U)                  # Non-dimensional
        # ω = ComputedField(((∂x(v) - ∂y(u) / f)/Rₒ))    # Non-dimensional and normalized
        # q = ComputedField(((f .+ ω)/h)/(f/H))          # Non-dimensional
        η = (h - H)/Δη                  # Non-dimensional
        u = (uh / h)/U                  # Non-dimensional
        v = (vh / h)/U                  # Non-dimensional
        ω = ((∂x(v) - ∂y(u) / f)/Rₒ)    # Non-dimensional and normalized
        q = ((f .+ ω)/h)/(f/H)          # Non-dimensional

BinaryOperation at (Center, Center, Center)
├── grid: 128×128×1 RectilinearGrid{Float64, Bounded, Bounded, Flat} on CPU with 3×3×0 halo
└── tree: 
    / at (Center, Center, Center)
    ├── / at (Center, Center, Center)
    │   ├── 129×129×1 Array{Float64, 3}
    │   └── 128×128×1 Field{Center, Center, Center} on RectilinearGrid on CPU
    └── 5.9782042352012025e-6

Pick a time-step that resolves the surface dynamics

In [5]:
wave_propagation_time_scale = model.grid.Δxᶜᵃᵃ / gravity_wave_speed
timestep = 0.1 * wave_propagation_time_scale
@printf("Timestep = [%6.4f] = [%6.4f] hours == [%6.4f] seconds.\n",timestep/timescale,timestep/3600,timestep)
simulation = Simulation(model, Δt = timestep, stop_iteration = Nt)

Timestep = [0.0008] = [0.0087] hours == [31.2256] seconds.


Simulation of ShallowWaterModel{RectilinearGrid{Float64, Bounded, Bounded, Flat, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, CPU}, CPU, Float64, WENO5{Float64, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing}, FPlane{Float64}, NamedTuple{(:uh, :vh, :h), Tuple{Oceananigans.Forcings.ContinuousForcing{Face, Center, Center, Nothing, Relaxation{Float64, typeof(Oceananigans.Forcings.onefunction), typeof(Oceananigans.Forcings.zerofunction)}, Tuple{Symbol}, Tuple{Int64}, Tuple{typeof(Oceananigans.Operators.identity1)}}, Oceananigans.Forcings.ContinuousForcing{Center, Face, Center, Nothing, Relaxation{Float64, typeof(Oceananigans.Forcings.onefunction), typeof(Oc

# Select output and run simulation

 We output the velocity field and free surface displacement, vorticity and PV.

In [6]:
output_fields = merge(model.solution, (η=η,u=u,v=v,ω=ω,q=q))
simulation.output_writers[:fields] = JLD2OutputWriter(model, output_fields,
                                                      schedule = IterationInterval(8),
                                                      prefix = "RSW_geostrophic_adjustment",
                                                      force = true)
@time run!(simulation)

341.633185 seconds (888.36 M allocations: 753.068 GiB, 40.56% gc time, 17.80% compilation time)


┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /Users/thaine1/.julia/packages/Oceananigans/PA1TE/src/Simulations/run.jl:168
┌ Info:     ... simulation initialization complete (26.451 seconds)
└ @ Oceananigans.Simulations /Users/thaine1/.julia/packages/Oceananigans/PA1TE/src/Simulations/run.jl:203
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /Users/thaine1/.julia/packages/Oceananigans/PA1TE/src/Simulations/run.jl:113
┌ Info:     ... initial time step complete (23.428 seconds).
└ @ Oceananigans.Simulations /Users/thaine1/.julia/packages/Oceananigans/PA1TE/src/Simulations/run.jl:120
┌ Info: Simulation is stopping. Model iteration 4096 has hit or exceeded simulation stop iteration 4096.
└ @ Oceananigans.Simulations /Users/thaine1/.julia/packages/Oceananigans/PA1TE/src/Simulations/simulation.jl:156


# Visualize the results

In [7]:
xη, yη = xnodes(η), ynodes(η)
videofile = "RSW_geostrophic_adjustment.mp4"
@printf("Making animation of time dependent solution:\nWriting output to [%s] with [%d] steps of length [%8.6f]s and final time = [%6.4f].\n",videofile,Nt,timestep,Nt*timestep)
file = jldopen(simulation.output_writers[:fields].filepath)
kwargs = (
          fillrange = true,
        levels = range(-2.2,2.2,length=64),
      colormap = :seismic
    )
iters = parse.(Int, keys(file["timeseries/t"]))

# Define observables:
iter = Observable(iters[1])
u = lift(iter) do iter
    u = file["timeseries/η/$iter"][:, :, 1]
end

tit_txt = lift(iter) do iter
    thistime = timestep*iter
    tit_txt = @sprintf("η at t = %4.2f = %3.1f hours",thistime/timescale,thistime/3600)    
end

# Make plot:
fig,ax,plt = contourf(xη / Lx, yη / Lx, u, figure = (resolution = (700, 600),),axis = (title = tit_txt, xlabel = L"x/L_x",ylabel = L"y/L_x",); kwargs...)
ax.aspect = DataAspect()
Colorbar(fig[1,2],plt)

# Control animation:
framerate = 30
record(fig,videofile, iters; framerate=framerate) do this_iter    
    iter[] = this_iter
end 

Making animation of time dependent solution:
Writing output to [RSW_geostrophic_adjustment.mp4] with [4096] steps of length [31.225605]s and final time = [127900.0781].


"RSW_geostrophic_adjustment.mp4"